# From Theory to Practice: Backtesting Our Quantile Idea

### So, Do Quantiles Actually Work?

In the last notebook, we came up with a neat idea: use quantiles to figure out a "probable" range for a candle's price movement. The logical next step is to ask the million-dollar question: can we actually make money with this?

To find out, I've used backtesting to see how different high/low percentile choices affect the win rate. Now, I understand this is a quiet, simple, and flawed representation of how the market works, but it's a starting point. And as we'll see, even simple models can show some interesting data. Let's build a strategy using the `backtesting.py` library and see if our theory holds any water.

In [1]:
# First, let's get the necessary tools for the job.
%pip install backtesting yfinance pandas

Note: you may need to restart the kernel to use updated packages.


### Building the strategy

Here's the game plan. We'll create a strategy that does the following for every candle:
1.  **Look Back:** It will look at all the *past* candles to calculate the percentage price moves (just like we did manually).
2.  **Calculate Quantiles:** Based on that history, it will calculate a quantile value (e.g., the 50th percentile) for both upward and downward moves. This gives us our dynamic thresholds.
3.  **Set TP and SL:** It will place a new trade using the current price and set a Take Profit (TP) and Stop Loss (SL) based on those quantile thresholds.

The beauty is that the TP and SL are no longer fixed numbers; they adapt as the bot sees more and more data. Let's code it up.

In [ ]:
import math
import pandas as pd
from backtesting import Strategy

class QuantileStrategy(Strategy):
    h_percentile = None  # Take Profit quantile
    l_percentile = None  # Stop Loss quantile
    mode = "long"        # "long" or "short"

    @staticmethod
    def default_func(series: pd.Series, q: float) -> pd.Series:
        """Expanding quantile of a series."""
        return series.expanding(min_periods=10).quantile(q)

    func = default_func

    def init(self):
        self.mode_coef = 1 if self.mode == "long" else -1
        self.mode_trade = self.buy if self.mode == "long" else self.sell

        o, h, l = self.data.Open, self.data.High, self.data.Low

        # Relative intrabar moves, shifted to avoid lookahead bias
        h_move = pd.Series((h - o) / o).shift(1)
        l_move = pd.Series((o - l) / o).shift(1)

        # Dynamic thresholds
        self.h_threshold = self.I(self.func, h_move, self.h_percentile) if self.h_percentile else None
        self.l_threshold = self.I(self.func, l_move, self.l_percentile) if self.l_percentile else None

    def next(self):
        if len(self.data) < 10:
            return

        price = self.data.Close[-1]
        h, l = self.h_threshold[-1], self.l_threshold[-1]

        if pd.isna(h) and pd.isna(l):
            return

        # If a percentile is None, set TP/SL to infinity
        tp = price * (1 + h * self.mode_coef) if h is not None else math.inf * self.mode_coef
        sl = price * (1 - l * self.mode_coef) if l is not None else -math.inf * self.mode_coef

        if self.position:
            self.position.close()

        # Sanity check: don't enter a trade if price is already outside SL/TP
        if sl * self.mode_coef < price * self.mode_coef < tp * self.mode_coef:
            self.mode_trade(
                tp=None if math.isinf(tp) else tp,
                sl=None if math.isinf(sl) else sl,
                size=1,
            )

### The Moment of Truth

Alright, the bot is coded. It's time to download some fresh data, plug in our strategy, and see how much theoretical money we made (or, more likely, lost). We'll start by setting our Stop Loss at the 50th percentile, meaning we're placing it at the *median* down move. For now, we'll leave the Take Profit open.

In [26]:
import yfinance as yf
from backtesting import Backtest

# Let's grab some data for Gold again.
tickers = ["GC=F"]
period = "1mo"
interval = "1h"

print("Fetching data for the grand experiment...")
data = yf.download(tickers=tickers, period=period, interval=interval, multi_level_index=False)

# Initialize the backtest environment.
bt = Backtest(data, QuantileStrategy, cash=10000)

# Run the backtest. We'll set the stop-loss percentile to 0.5 (the 50% quantile).
result = bt.run(h_percentile=0.01, l_percentile=0.56)

print("\n--- Backtest Results ---")
print(result)

Fetching data for the grand experiment...


/tmp/ipykernel_11855/1916933722.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=tickers, period=period, interval=interval, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


--- Backtest Results ---
Start                     2025-08-19 22:00...
End                       2025-09-19 20:00...
Duration                     30 days 22:00:00
Exposure Time [%]                     2.96443
Equity Final [$]                   9989.57052
Equity Peak [$]                       10000.0
Return [%]                           -0.10429
Buy & Hold Return [%]                10.43021
Return (Ann.) [%]                    -0.93475
Volatility (Ann.) [%]                 0.23574
CAGR [%]                             -0.84694
Sharpe Ratio                          -3.9652
Sortino Ratio                        -3.87178
Calmar Ratio                         -8.96255
Alpha [%]                            -0.10987
Beta                                  0.00053
Max. Drawdown [%]                    -0.10429
Avg. Drawdown [%]                    -0.10429
Max. Drawdown Duration       30 days 11:00:00
Avg. Drawdown Duration       30 days 11:00:00
# Trades                                   15
Win Rate

### The Verdict and What's Next

So, there are the results. Genius strategy or a spectacular failure? The numbers speak for themselves. 

This simple backtest is made as a foundation for higher research and testing. 
We can now build upon this strategy in many ways:

*   **Different Calculation Methods:** Instead of an `expanding` window, we could use a `rolling` window to only consider the last N candles.
*   **Smarter Filtering:** We could filter the historical data to only calculate quantiles during high-volatility periods, for example.
*   **Optimization:** We can run this backtest hundreds of times to find the *optimal* quantile percentages for TP and SL. (good foreshadowing)

I would not consider this perfect, but it seems like an okay start. We've successfully turned a statistical idea into a testable hypothesis, which is a step forward.